In [ ]:
!pip install python-geohash
!pip install pygeohash
!pip install fastdtw
!pip install numba
!pip install statsmodels

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install --upgrade pyarrow --only-binary :all:

In [57]:

import imp
import pdb
from datetime import datetime
import pandas as pd

# Import our dependency
from read_data import *
from social_group import *
from dtw_fun import *
from network_filtering import *
from group_network import *
import warnings
warnings.filterwarnings("ignore")

# Data path
base_path = 'lee_county_monthly2/lee_county_202307.csv'

# Define the starting month of the whole project. 2022-08-01 to 2023-07-31
# original_date=datetime(2022, 8, 1)

# Define the pre disaster date range
start_date = datetime(2023, 7, 1)

# 1. Filter Holiday records from 2019-12-24 18:00 to 22:00
start_time = pd.Timestamp("2022-12-24 18:00:00")
end_time   = pd.Timestamp("2022-12-24 22:00:00")

# Select the location you want
# somewhere in Maryland, Florida, NYC, Longisland, A small county in the middle of Long Island, Region around Brookhaven and Mastic Beach
selected_region = 'Lee_county'  


In [58]:
'''
1. Read Data
'''
# read_save_all_data(start_date, end_date, base_path)

filtered_df, min_lat, max_lat, min_lon, max_lon=read_region_data(selected_region, base_path)


'''
2. Filter local resident and residents with minimmun appearance in month
'''
filtered_df=apply_residency_filter(filtered_df, min_lat, max_lat, min_lon, max_lon)
linked_df = apply_residency_filter_appear(filtered_df, min_appearances=5)




Original DataFrame shape: (110121, 9)


In [59]:
'''
3. Identify Social Group
'''
user_group_df, result_df_with_group=user_group_links(start_date, linked_df, min_appear=10)


    device_id            utc_timestamp_1            utc_timestamp_2  \
0  7454090129  2023-07-01T08:03:37-04:00  2023-07-01T08:22:01-04:00   
1  7454090129  2023-07-01T11:09:10-04:00  2023-07-01T12:49:30-04:00   
2  5815618791  2023-07-01T11:02:37-04:00  2023-07-01T11:34:50-04:00   
3  6459566408  2023-07-01T17:50:16-04:00  2023-07-01T18:24:41-04:00   
4  6459566408  2023-07-01T18:33:02-04:00  2023-07-01T18:53:33-04:00   

   latitude_1  longitude_1  geohash_1  latitude_2  longitude_2  geohash_2  \
0   26.562816   -81.984728  dhtxn7se9   26.681269   -81.995024  dhtxy1jj9   
1   26.629715   -82.016635  dhtxmxv33   26.479828   -81.807639  dhtydkt2p   
2   26.606348   -81.978571  dhtxqe910   26.673228   -81.932356  dhtxxxdwv   
3   26.599014   -82.032083  dhtxm64n3   26.608150   -82.023561  dhtxmeb78   
4   26.608150   -82.023561  dhtxmeb78   26.608397   -82.023569  dhtxmebm0   

                 unix_time_1  ...          timestamp_1          timestamp_2  \
0  2023-07-01T08:03:37-04:00  .

In [60]:
'''
4. Identify Social Links using DTW
'''
df_dtw_results=dtw_compute(result_df_with_group,start_time, end_time, min_trj_len = 10)



Number of active users: 208426
Fast-DTW running …


100%|██████████| 6174/6174 [00:07<00:00, 798.64it/s] 


Threshold selected: 1.3453468084335327
Number of candidate pairs predicted as social links: 1853
Records during holiday: 0
Number of holiday bonding nodes area: 0
Number of holiday bonding user pairs (edges): 0
DTW Prediction details saved to Individual_social_network.csv


In [61]:
'''
5. Network Filtering
'''
graph_df=filter_network_eadm(df_dtw_results, linked_df, start_date)

Scanning groups: 100%|██████████| 55927/55927 [00:05<00:00, 9795.10it/s] 


In [62]:
'''
6. Social Group Network
'''
group_network_df=build_group_network(graph_df, user_group_df, start_date)

Total individual links (predicted connected): 1853
Number of group candidate pairs: 1671
All Group 2655
Unique Device Id count: 2981
Group that have links: 995
